# Images

This page focuses on the features associated with docker images.

## Build image

You can build docker images using the `docker build <path to the folder>` command. The `<folder path>` should contain a `dockerfile` with the instructions to build the images.

---

Consider example - the following cell creates the simplest possible `dockerfile` it just modifies alpine linux to echo message `this_is_new_container` at container startup.

In [6]:
cat << 'EOF' > images_files/dockerfile

FROM alpine
CMD ["echo", "this_is_new_image"]

EOF

**Note:** Even if the `Dockerfile` is located in the current working directory, you still need to specify the `path to folder` in the `docker build` command. In such cases, you should use the command `docker build .`.

The following example demonstrates the construction of a Docker image. It employs `images_files` as the working directory to emphasize the need for explicitly specifying `.` as the path.

Also here is specified `-t docker_build_example` option it specifies name of the container. 

In [8]:
cd images_files
docker build -t docker_build_example .
cd ..

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  2.048kB
Step 1/2 : FROM alpine
 ---> a606584aa9aa
Step 2/2 : CMD ["echo", "this_is_new_image"]
 ---> Running in 441872d8a1e9
Removing intermediate container 441872d8a1e9
 ---> cc90f738517a
Successfully built cc90f738517a
Successfully tagged docker_build_example:latest


After executing the previous cell, you'll be able to find the `docker_build_example` image in the list of images.

In [10]:
docker images | grep docker_build_example

docker_build_example          latest    cc90f738517a   3 minutes ago   7.8MB


Ok, finally, let's try to run container based on the image we created, to make sure that docker completes the instructions specified in the `dockerfile`.

In [13]:
docker run --rm docker_build_example
docker rmi docker_build_example

this_is_new_image
Untagged: docker_build_example:latest
Deleted: sha256:cc90f738517a2445baa3b8da64400a7d0ad79beabc7c091e9d0f513cab94c759


## Commit container to image

The `docker commit` command allows you to save the current state of a container as a new image. This is useful if you have made changes or created something new within a container and want to preserve these modifications. To save the current state of a container as a new image, use the following syntax:

```
docker commit <container name> <new image name>
```

---

As an example, let us run Alpine Linux and try to ask it Python version.

In [36]:
docker run --rm --name commit_container -itd alpine
docker exec commit_container python3 --version

7a095fff8ce568c6a238676ee753d8c99ed99705b89dded46daa52682675cd69
OCI runtime exec failed: exec failed: unable to start container process: exec: "python3": executable file not found in $PATH: unknown


: 126

As a result, we got an error. No big deal - we could install Python on any system.

In [37]:
docker exec commit_container apk add python3 &> /dev/null
docker exec commit_container python3 --version

Python 3.12.3


After installation we have the Python version. Now by using `commit` we "freeze" such state of the container as the image. And any container created on from this image will be just like alpine linux but with installed python.

In [33]:
docker commit commit_container alpine_with_python
docker run --rm -itd --name al_py alpine_with_python
docker exec al_py python3 --version

7a40579ac221bcc570751f0ab496e10291134448f5885e7c614d0544b21c69a1
Python 3.12.3


So containers that use the `alpine_with_python` image as a base will have the python version by default.

In [39]:
docker stop commit_container al_py
docker rmi alpine_with_python

commit_container
Error response from daemon: No such container: al_py
Untagged: alpine_with_python:latest
Deleted: sha256:909728da78e4e6032ade63f135f729f85075df783e86c72688e20f152f5d27ec
Deleted: sha256:d45b95ae0c5e8765eebc035f638f39e1cfcf6d20cd0df26e0915f9ab087793c5


## Dockerfile directives

Dockerfile directives describe the behaviour of the dockerbuild command.

| Directive        | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `FROM`           | Sets the base image for the new image.                                      |
| `WORKDIR`        | Sets the working directory within the image.                                |
| `COPY`           | Copies files or directories from the host to the image.                     |
| `RUN`            | Executes commands in a new layer on top of the current image.               |
| `CMD`            | Provides a command that will run when a container is started.               |
| `EXPOSE`         | Informs Docker that the container listens on the specified network ports.   |
| `ENV`            | Sets environment variables.                                                 |
| `ADD`            | Copies files, directories, or remote file URLs from the host to the image.  |
| `ENTRYPOINT`     | Configures a container to run as an executable.                             |
| `VOLUME`         | Creates a mount point with specified path and marks it as holding external data. |
| `USER`           | Sets the username or UID to use when running the image.                     |
| `LABEL`          | Adds metadata to an image.                                                  |
| `ARG`            | Defines a variable that users can pass at build-time to the builder.        |
| `ONBUILD`        | Adds a trigger instruction to be executed when the image is used as a base for another build. |
| `STOPSIGNAL`     | Sets the system call signal that will be sent to the container to exit.     |
| `HEALTHCHECK`    | Tells Docker how to test a container to check that it is still working.     |
| `SHELL`          | Allows the default shell used for the shell form of commands to be overridden. |

**Check more** in [specific page](images/docker_file_directives.ipynb) for dockerfile directives.